In [1]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [2]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [3]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight).

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [4]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [5]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [6]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [7]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [8]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [9]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [10]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [11]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [12]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [13]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [14]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [15]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [16]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [17]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [18]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [19]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [20]:
Level=8;

### Calculate the Proficiency Bonus

In [21]:
ProficiencyBonus=1+np.ceil(Level/4)

In [22]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [23]:
OtherBonus=1; #Rod of pact keeper

In [24]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [25]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [26]:
WeaponBonus=1; #Magical +1 weapon

In [27]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [28]:
MeleeHit

8

In [29]:
### Calculate Hit for Weapon Attacks with Dexterity

In [30]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [31]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [32]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [33]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [34]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [35]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [36]:
PrMiss=(OAC- SpellHit -1)/20 

In [37]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [38]:
Ad=2

In [39]:
PrHitA=1-(PrMiss)**(Ad)

In [40]:
PrHitA

0.91

#### testing function

In [41]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [42]:
Beams=min(math.floor(Level/5)+1,4)

In [43]:
Beams

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [44]:
AverageAgonisingBlastDamage=5.5

In [45]:
AverageDamagePerBeam=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [46]:
AverageDamagePerBeam

9.5

Additional modifier for Hex spell

In [47]:
HexAverageDamage=3.5

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [48]:
Critit=1/20;

In [49]:
2 * PrHit * PrMiss * AverageDamagePerBeam+ 2* (Critit) *(1-Critit) * AverageDamagePerBeam+ PrHit**2 * 2* AverageDamagePerBeam + 2* (Critit) *(Critit) * AverageDamagePerBeam

14.249999999999998

In order to easily generalise to higher number of Beams we use the binomial distribution. we check for 2 Beams to see the same result as above.

Without taking into account Crit Hit

In [53]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerBeam + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerBeam 

In [54]:
ADNCR

13.3

Calculating Crit Hit

In [55]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerBeam + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerBeam 

In [56]:
ACHD

0.9500000000000003

Adding the two together

In [57]:
TAD=ADNCR+ACHD

In [58]:
TAD

14.250000000000002

#### Calculating for arbitrary number of Beams

In [59]:
TotalDamage=0;

In [60]:
indexBeam = 1
while indexBeam < (Beams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,Beams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,Beams,Critit)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [61]:
TotalDamage

14.25

### Average damage with  Advantage

In [62]:
n=1

In [68]:
TotalBeams=n*Beams

In [69]:
CrititA=1-(1-1/20)**2

In [70]:
TotalDamage=0

In [71]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [72]:
TotalDamage

19.142500000000005

### Average damage with Advantage and Hex

In [73]:
n=1

In [74]:
TotalBeams=n*Beams

In [75]:
TotalDamage=0

In [76]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)
    indexBeam += 1

In [77]:
TotalDamage

26.195000000000004

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [78]:
n=2;

In [79]:
TotalBeams=n*Beams

In [80]:
TotalDamage=0

In [81]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHit)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,Critit)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [82]:
TotalDamage

28.500000000000007

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [83]:
n=2;

In [84]:
TotalBeams=n*Beams

In [85]:
TotalDamage=0

In [86]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*AverageDamagePerBeam+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*AverageDamagePerBeam
    indexBeam += 1

In [87]:
  TotalDamage

38.285000000000004

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [88]:
n=2;

In [89]:
TotalBeams=n*Beams

In [90]:
TotalBeams

4

In [91]:
TotalDamage=0

In [92]:
indexBeam = 1
while indexBeam < (TotalBeams+1):
    TotalDamage+=scipy.stats.binom.pmf(indexBeam,TotalBeams,PrHitA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)+scipy.stats.binom.pmf(indexBeam,TotalBeams,CrititA)*indexBeam*(AverageDamagePerBeam+HexAverageDamage)
    indexBeam += 1

In [93]:
TotalDamage

52.39

#### Testing the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [95]:
repetitions=4;average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [96]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

52.39

### Without Action surge, only advantage with two eldrich blast rays (repetitions=2)

In [98]:
repetitions=Beams;average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [99]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

26.195

### With Action Surge and spirit shroud so that I can use the celestial warlock's bonus to radient damage.

In [100]:
spirit_shroud=4.5

In [101]:
radient_damage_bonus=Modifiers[Abilities.index('CHA')]# since it is applied only to the first beam, multiply with factor 1/2 to calculate average damage

In [102]:
repetitions=4; average_damage_per_hit=AverageDamagePerBeam+spirit_shroud++HexAverageDamage+radient_damage_bonus/2;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [103]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

78.585

#### Assuming fiend warlock (no radient bonus, but darkness as a bonus spell). Hence I have one spell slot to use for Hex 

In [104]:
repetitions=4; average_damage_per_hit=AverageDamagePerBeam+HexAverageDamage+spirit_shroud;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [105]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

70.525

## Paladin built: calculate the damage of Melee (+1 weapon) + Hex+ Green Flame Blade+ Radient damage bonus + two Smites with Advantage

#### Magical +1 Long sword damage 1d8+1 (+4) + Green flame blade (calculating for 2 targets) 

In [106]:
WeaponAverageDamage=4.5+1+Modifiers[Abilities.index('STR')]+2*4.5+4

In [107]:
WeaponAverageDamage

22.5

In [108]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [109]:
PrHit

0.7

In [110]:
PrMiss=1-PrHit

In [111]:
Ad=2

In [112]:
PrHitAd=1-(PrMiss)**(Ad)

In [113]:
PrHitAd

0.9099999999999999

#### Divine smite (3rd level slot) 4d8 average 4*4.5

In [114]:
Smite=4.5*4

#### bonus smite: I am assuming  searing smite and a celestial warlock's bonus to radient damage

In [115]:
BonusSmite=3*3.5+Modifiers[Abilities.index('CHA')]

In [116]:
HexAverageDamage=3.5 # I will cast it with a 1st level paladine slot

In [117]:
TotalDamage= (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [118]:
TotalDamage

58.93874999999999

#### Using the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [119]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [120]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

58.93875

#### assuming non-celestial warlock

In [121]:
BonusSmite=3*3.5

In [122]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [123]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

54.90875

## Higher than the fighter (when Eldrich Blast only is considered) but asks for the consumption of two spell slots of the highest level (+1 first level). The characters casts first searing smite as bonus action. Slightly more complecated mechanics. If the Fighter combo consumes one slot with spirit shroud (radient) and one with Hex then I can exploit the Celestial Warlock Bonus and surpass the Paladin combo. At this level, I would not have another slot as a Paladin to also cast spirit shroud. The fighter combo suprasses the paladin combo at this level also without the Celestial Warlock Bonus. 